# Module 9. Natural language processing
# Topic modeling

## Lecture objectives
1. Provide practice with importing PDFs and handling exceptions
2. Demonstrate how to estimate a topic model using LDA
3. Discuss how to adjust the model parameters in the search for a meaningful topic model

So far, we've got some text into Python from a PDF, cleaned it up, and split the string into sentences and words. We've also done simple word counts. Now, we'll see what topic modeling - one method under the broader umbrella of natural language processing - can do.

Two examples in urban studies that use topic modeling are [Han et al.](https://www.tandfonline.com/doi/full/10.1080/01944363.2020.1831401) who analyze election communications, and [Brinkley & Stahmer](https://journals.sagepub.com/doi/abs/10.1177/0739456X21995890) who analyze General Plans. But let's say a little about the principles first.

Topic modeling is a method to identify patterns in text documents. It's an *unsupervised* machine learning technique, i.e. it doesn't need to be "trained" on a dataset of known topics. Rather, it identifies topics from the ground up. 

In that sense, topic modeling is related to cluster analysis. However, a document can belong (partially) to multiple topics, where an observation is typically assigned to a single cluster.

But both cluster analysis and topic modeling perform *dimensionality reduction*. Suppose we have 1000 distinct words in a document, that we reduce to 10 topics. We are going from a 1000-dimensional space to a 10-dimensional space. Obviously, 10 dimensions are much easier to interpret.

There are several algorithms that implement topic modeling; we'll focus on Latent Dirichlet Allocation (LDA) (as in the Han et al. paper). We'll use the `gensim` library, although `sklearn` is a popular alternative. `gensim` is dedicated to NLP, while `sklearn` is a more general machine-learning library.

We'll start with looking at a selection of Climate Action Plans (CAPs) from cities and counties in California. These are some of the plans I analyzed [in a project on equity in CAPS](https://journals.sagepub.com/doi/10.1177/0739456X211072527) with Hillary Angelo, Key MacFarlane, and James Sirigotis. Thanks to these collaborators for permission to share this dataset.

The plans are in a directory in your GitHub repository. We can see the list using `os`.

In [1]:
import os
filelist = os.listdir('../data/CAPs')  

# Look at the first 10
filelist[:10]

['palm_springs-_climate_action_plan.pdf',
 'Hillsborough_GHG Emissions Inventory_Climate Action Plan.pdf',
 'Mill Valley Climate Action Plan.pdf',
 'Palo Alto_CPP.pdf',
 'Redwood City_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_General Plan Policy_General Plan Implementation Measures.pdf',
 'Martinez_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_General Plan Policy.pdf',
 'Emeryville_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan.pdf',
 'Merced_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_General Plan Policy_General Plan Implementation Measures.pdf',
 'Cathedral City_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_Energy Action Plan.pdf',
 'San Leandro_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_General Plan Policy_General Plan Implementation Measures_Codes or Ordinances.pdf']

Rather than writing code to read each file individually, let's write a function which takes a filename, and returns the text of that file. We've seen all this code before.

We can then create a list of documents. Each element of that list will be the text of a single CAP.

In [2]:
from pdfminer.high_level import extract_text
import re

def readPDF(filename):
    txt = extract_text('../data/CAPs/'+filename)
    
    # use regex to remove punctuation, numbers, etc.
    txt = re.sub(r"[^A-z\s]", "", txt)
    # and to remove whitepace
    txt = re.sub(r"\s+", " ", txt) 
    
    print('Finished {}'.format(filename))
    return txt

# use a list comprehension to read in all the files
# only do those that end with .pdf
caps = [readPDF(fn) for fn in filelist if fn.endswith('.pdf')]

Finished palm_springs-_climate_action_plan.pdf
Finished Hillsborough_GHG Emissions Inventory_Climate Action Plan.pdf
Finished Mill Valley Climate Action Plan.pdf
Finished Palo Alto_CPP.pdf
Finished Redwood City_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_General Plan Policy_General Plan Implementation Measures.pdf
Finished Martinez_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_General Plan Policy.pdf
Finished Emeryville_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan.pdf
Finished Merced_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_General Plan Policy_General Plan Implementation Measures.pdf
Finished Cathedral City_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_Energy Action Plan.pdf
Finished San Leandro_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_General Plan Policy_General Plan Implementation Measures_Codes or Ordinances.pdf
Finished Santa Cruz_Vulnerability Assessment_GHG Emissions

Let's look at some random parts of a couple of plans to make sure they loaded.

In [3]:
print(caps[0][10000:10200])
print(caps[2][10000:10200])

ws the trajectory the City will follow given population growth and full implementation of state and federal emissions reduction programs such as auto emissions standards and requirements for renewable
ential and commercial so lar development over years Assembly Bill AB This bill allows Cali fornia municipalities to help citizens nance renewable and energy ef ciency projects by issuing a bond to pay


Now, as before, let's remove the stop words. 

We'll use a list comprehension again, but this time in a nested way.  In the outer list, we loop over plans. In the inner list, we loop over words in that plan.

We could have done this another way through adding to the `readPDF` function, and making that return a list of words that exclude stop words.

At this point, we might want to lemmatize as well, but we'll skip that here.

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

swords = stopwords.words('english')
wordlists = [[word for word in word_tokenize(cap.lower()) 
                 if word not in swords] 
                 for cap in caps]

So for each plan, we have a list of words. For example, the first ten words from the first plan are as follows.

In [7]:
wordlists[0][:10]

['table',
 'contents',
 'executive',
 'summary',
 'energy',
 'efficiency',
 'climate',
 'action',
 'targets',
 'summary']

Which makes sense when we look at the start of that plan.

In [8]:
caps[0][:100]

' Table of Contents I Executive Summary Energy Efficiency Climate Action Targets Summary of Costs and'

We are at the point where we can do topic modeling.

The `gensim` documentation is pretty thorough.

Note that there are lots of options. The most important are:
* `corpus`: the text. More on the format for this below
* `num_topics`: how many topics you want to identify
* `alpha`: the expected distribution of topics across documents (i.e., are topics concentrated in a few documents)
* `eta`:  (sometimes called `beta`): the expected distribution of words across topics (i.e., are words concentrated in a few topics) 

However, sensible defaults are provided. So normally, a good approach is to start with the defaults and adjust accordingly. 

In [9]:
import gensim
help(gensim.models.LdaMulticore)

Help on class LdaMulticore in module gensim.models.ldamulticore:

class LdaMulticore(gensim.models.ldamodel.LdaModel)
 |  LdaMulticore(corpus=None, num_topics=100, id2word=None, workers=None, chunksize=2000, passes=1, batch=False, alpha='symmetric', eta=None, decay=0.5, offset=1.0, eval_every=10, iterations=50, gamma_threshold=0.001, random_state=None, minimum_probability=0.01, minimum_phi_value=0.01, per_word_topics=False, dtype=<class 'numpy.float32'>)
 |  
 |  An optimized implementation of the LDA algorithm, able to harness the power of multicore CPUs.
 |  Follows the similar API as the parent class :class:`~gensim.models.ldamodel.LdaModel`.
 |  
 |  Method resolution order:
 |      LdaMulticore
 |      gensim.models.ldamodel.LdaModel
 |      gensim.interfaces.TransformationABC
 |      gensim.utils.SaveLoad
 |      gensim.models.basemodel.BaseTopicModel
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, corpus=None, num_topics=100, id2word=None, workers=

[The `gensim` documentation](https://radimrehurek.com/gensim/models/ldamulticore.html) provides some simple examples that help with getting up and running. There's also many other examples online. 

Conceptually, we need to:
* Convert our text to a list of lists. The outer list is each plan, and the inner list is words in that plan. We did this already.
* Create a gensim Dictionary (basically, each word gets an integer id)
* Feed that to the `gensim.models.LdaMulticore` function

The tricky part here is getting the data into the format (e.g. a list of words, list of word ids, string) that `gensim` expects. I did this mostly by adapting the examples on the `gensim` website.

Here, we choosing 5 topics - an arbitrary number. We aren't specifying the `alpha` and `eta` hyperparameters. Rather, we are using the `gensim` defaults.

In [10]:
dictionary = gensim.corpora.Dictionary(wordlists)
corpus = [dictionary.doc2bow(wl) for wl in wordlists]
# LdaMulticore uses multiple cores (thus, it runs faster)
# If you have problems, try replacing LdaMulticore with LdaModel
model = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=5)

The `model` object now contains the results of our LDA model. We can explore some of its attributes and functions. The [documentation](https://radimrehurek.com/gensim/models/LdaModel.html#module-gensim.models.LdaModel) is also fairly comprehensive.

In [11]:
model.show_topics()

[(0,
  '0.014*"city" + 0.010*"emissions" + 0.010*"climate" + 0.010*"energy" + 0.008*"plan" + 0.007*"e" + 0.007*"action" + 0.007*"ghg" + 0.006*"use" + 0.006*"community"'),
 (1,
  '0.013*"emissions" + 0.013*"energy" + 0.012*"e" + 0.009*"climate" + 0.009*"city" + 0.009*"n" + 0.008*"plan" + 0.007*"action" + 0.007*"r" + 0.007*"c"'),
 (2,
  '0.013*"energy" + 0.013*"emissions" + 0.013*"city" + 0.011*"climate" + 0.009*"plan" + 0.007*"e" + 0.007*"ghg" + 0.006*"action" + 0.006*"use" + 0.006*"reduction"'),
 (3,
  '0.105*"cid" + 0.012*"energy" + 0.012*"emissions" + 0.010*"city" + 0.009*"e" + 0.008*"cidcidcidcid" + 0.008*"plan" + 0.007*"n" + 0.007*"climate" + 0.007*"r"'),
 (4,
  '0.141*"cid" + 0.012*"emissions" + 0.010*"climate" + 0.010*"cidcidcidcid" + 0.009*"energy" + 0.009*"city" + 0.008*"plan" + 0.008*"cidcidcid" + 0.007*"cidcidcidcidcid" + 0.007*"e"')]

In [12]:
model.show_topic(0)

[('city', 0.014039329),
 ('emissions', 0.010445332),
 ('climate', 0.010128679),
 ('energy', 0.009967672),
 ('plan', 0.008443469),
 ('e', 0.0072769285),
 ('action', 0.0069962414),
 ('ghg', 0.0068504084),
 ('use', 0.006442036),
 ('community', 0.0058960253)]

## Troubleshooting
So it looks like we have some topics. But there is a lot of nonsense. 

First of all, there are some one and two letter words.

Second, what is `cidcid`? 

Let's troubleshoot. We can find a plan that has this string in it.

In [13]:
for cap in caps:
    if 'cidcid' in cap:
        # stop once we find that
        break
        
# look at the start of that plan
cap[:6000]

'Climate Action Plan Climate Action Plan Introduction Purpose Existing Conditions Climate Action Goals Policies and Programs Climate Change Reducing Carbon Footprint Climate Change Adaptation Zero Waste Additional CityRelated Emission Reduction Programs Land Use Mobility Natural Environment Community Vitality Safety Noise Appendix Emission Reduction Targets based on programs Introduction The Mill Valley Climate Action Plan was adopted as part of the Mill Valley General Plan October Language contained in this Climate Action Plan are excerpts from the larger Mill Val ley General Plan that is available at wwwcityofmillvalleyorg generalplan Purpose Climate change is caused by an increase in the concentration of atmospheric greenhouse gases Potential climate change impacts in Northern California include declining water supplies spread of disease diminished agricultural productivity sea level rise and in creased incidence of wild re ooding and landslides Like many communities Mill Valley is 

Who knows what caused that. PDFs are hard to parse!

So let's drop everything from our wordlist that is:
* Less than 3 letters
* Has `cid` in it

We can add these conditions. Note that we check separately to see if the word is equal to `cid`, or if multiple cids are in the word. This avoids dropping words like "incidental."

In [14]:
swords = stopwords.words('english')
wordlists = [[word for word in word_tokenize(cap.lower()) 
                 if word not in swords and len(word)>2 and word!='cid' and 'cidcid' not in word] 
                 for cap in caps]

And estimate our topic model again.

In [15]:
dictionary = gensim.corpora.Dictionary(wordlists)
corpus = [dictionary.doc2bow(wl) for wl in wordlists]
model = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=5)
model.show_topics()

[(0,
  '0.012*"city" + 0.012*"emissions" + 0.012*"energy" + 0.012*"climate" + 0.010*"plan" + 0.008*"ghg" + 0.007*"action" + 0.007*"reduction" + 0.007*"water" + 0.006*"program"'),
 (1,
  '0.018*"emissions" + 0.016*"energy" + 0.012*"climate" + 0.012*"plan" + 0.011*"city" + 0.008*"action" + 0.007*"reduction" + 0.006*"program" + 0.006*"ghg" + 0.006*"water"'),
 (2,
  '0.015*"city" + 0.014*"emissions" + 0.013*"climate" + 0.013*"energy" + 0.009*"plan" + 0.008*"action" + 0.007*"gas" + 0.007*"reduction" + 0.006*"ghg" + 0.006*"waste"'),
 (3,
  '0.014*"city" + 0.013*"emissions" + 0.013*"energy" + 0.009*"reduction" + 0.008*"plan" + 0.008*"ghg" + 0.008*"climate" + 0.007*"action" + 0.007*"waste" + 0.007*"community"'),
 (4,
  '0.014*"emissions" + 0.013*"energy" + 0.013*"city" + 0.009*"plan" + 0.009*"climate" + 0.007*"action" + 0.007*"ghg" + 0.006*"water" + 0.006*"use" + 0.006*"gas"')]

This looks better! 

## Visualizing topic models
How can we visualize the topics?

The data format looks pretty straightforward. Each topic is a list of tuples of (word, weight). With some effort, we might be able to plot this ourselves. 

But a quick web search reveals that there is a [Python library specificially designed to visualize LDA outputs](https://pyldavis.readthedocs.io/en/latest/readme.html)! 

In [16]:
import pyLDAvis
import pyLDAvis.gensim_models   # note that in previous versions this was called pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model, corpus, dictionary)

/Users/chaithra/opt/anaconda3/envs/uds/lib/python3.10/site-packages/pyLDAvis/_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.004148  0.002112       1        1  36.562508
2      0.000849  0.001145       2        1  22.114721
0     -0.000211 -0.002400       3        1  19.298198
3     -0.004827  0.002097       4        1  17.197890
4      0.000041 -0.002954       5        1   4.826683, topic_info=            Term         Freq        Total Category  logprob  loglift
350         city  5741.000000  5741.000000  Default  30.0000  30.0000
734       energy  6257.000000  6257.000000  Default  29.0000  29.0000
717    emissions  6810.000000  6810.000000  Default  28.0000  28.0000
1586        plan  4645.000000  4645.000000  Default  27.0000  27.0000
355      climate  5203.000000  5203.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
1280       local    81.024074  1931.168378   Topic5  -5.5940  -0.1401
1705      public    68.959623  1442.471783   Topic5  -5.7553  -0.0096
697   efficiency    72.782225  1746.381982   Topic5  -5.7013  -0.1468
966        green    72.506116  1800.470830   Topic5  -5.7051  -0.1811
968   greenhouse    73.181468  1920.160133   Topic5  -5.6958  -0.2362

[457 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
6398       1  0.574704  abandonment
6398       2  0.191568  abandonment
6398       3  0.191568  abandonment
6398       4  0.191568  abandonment
11932      1  0.481935   abelvedere
...      ...       ...          ...
2432       1  0.330001        years
2432       2  0.199208        years
2432       3  0.150915        years
2432       4  0.255550        years
2432       5  0.064390        years

[1355 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1, 4, 5])

How do we interpret the visualization? You can find a detailed explanation [here](https://aclanthology.org/W14-3110.pdf). In short:
* The size of each circle is proportional to the frequency of that topic
* The distance between circles shows how closely they are related
* The bars represent the overall frequency of a word (blue) and the frequency within that topic
* The "relevance metric" slider controls the sorting. The default is to sort by overall frequency within a topic. As you slide to the left, words that are disproportionately frequent in that topic rise to the top.

## Making sense of topic models
So what does all this mean?

This is where exploratory analysis comes in. We probably want to adjust the number of topics and the parameters, until we find a set of topics that makes intuitive sense.

If we were doing regression analysis, this would be "fishing" (bad!). But in unsupervised machine learning, this type of exploration is an inherent part of the process. We aren't testing hypotheses, just searching for patterns and understanding.

Remember that the `LdaMulticore` function takes the `num_topics`, `alpha` and `eta` parameters. We could adjust those. We might also want to exclude certain uninformative words based on our present context — perhaps "city" or "climate."

`alpha` controls the expected distribution of topics across documents. A higher value of `alpha` means that each document is expected to contain more of a mix of topics, rather than focusing on a few topics.

`eta` (sometimes called beta) controls the expected distribution of words across topics. A higher value of `eta` means that topics are more similar in terms of their mixture of words.

For climate plans, let's assume that documents have more of a mix of topics, so we'll set `alpha=0.9`. For `eta`, it's unclear what to expect, but let's try `eta=0.5` for now.

In [17]:
words_to_exclude = ['city','climate','action','emissions','ghg','reduction','plan']

wordlists = [[word for word in word_tokenize(cap.lower()) 
                 if word not in swords and word not in words_to_exclude
                     and len(word)>2 and word!='cid' and 'cidcid' not in word] 
                 for cap in caps]
dictionary = gensim.corpora.Dictionary(wordlists)
corpus = [dictionary.doc2bow(wl) for wl in wordlists]

model = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=5, alpha = 0.7, eta=0.5)
pyLDAvis.gensim_models.prepare(model, corpus, dictionary)

/Users/chaithra/opt/anaconda3/envs/uds/lib/python3.10/site-packages/pyLDAvis/_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.001509 -0.000923       1        1  34.380808
4     -0.000731 -0.001549       2        1  27.180293
3      0.000811  0.001347       3        1  14.074459
2      0.000563  0.000585       4        1  12.197697
0     -0.002152  0.000540       5        1  12.166743, topic_info=             Term         Freq        Total Category  logprob  loglift
730        energy  5752.000000  5752.000000  Default  30.0000  30.0000
2368        water  2414.000000  2414.000000  Default  29.0000  29.0000
916           gas  2524.000000  2524.000000  Default  28.0000  28.0000
2295          use  2493.000000  2493.000000  Default  27.0000  27.0000
1664      program  2317.000000  2317.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
332        change   159.620483  1376.738149   Topic5  -5.7650  -0.0482
961         green   177.462099  1693.659362   Topic5  -5.6591  -0.1494
400     community   203.377915  2280.746812   Topic5  -5.5227  -0.3107
1834  residential   155.996235  1413.499185   Topic5  -5.7880  -0.0975
2421         year   156.450978  1426.690663   Topic5  -5.7851  -0.1039

[444 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
7654       1  0.297792   accpp
7654       2  0.297792   accpp
7654       3  0.148896   accpp
7654       4  0.148896   accpp
7654       5  0.148896   accpp
...      ...       ...     ...
11209      1  0.362301  yields
11209      2  0.271726  yields
11209      3  0.181150  yields
11209      4  0.090575  yields
11209      5  0.181150  yields

[1387 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 4, 3, 1])

That gave some insights into what types of topics were included in plans, but it was hard to find a meaningful group of topics. We could experiment further with the hyperparameters. But it could also be a limitation of the small number of plans (30) in our sample. With a few hundred plans, we might generate further insights.

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Experiment with the parameters (and perhaps the list of words that you exclude), and see if you can come up with sensible topics.
</div>

<div class="alert alert-block alert-info">
<h3>Key Takeaways</h3>
<ul>
  <li>Topic modeling is a form of dimensionality reduction. The aim is to make your data easier to interpret.</li>
  <li>Exploration and iteration are the keys.</li>
  <li>After your first set of results, maybe there are texts or words that you want to exclude.</li>
  <li>Then, experiment with adjusting the parameters.</li>
    <li>Success is defined based on whether you find the results useful!</li>
</ul>
</div>